<a href="https://colab.research.google.com/github/jayy1809/IP-Patenting-CrewAI/blob/main/IP_crew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install crewai crewai-tools

  Using cached langchain_core-0.1.52-py3-none-any.whl (302 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.9
    Uninstalling langchain-core-0.2.9:
      Successfully uninstalled langchain-core-0.2.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 1.0.7 requires langchain-core<0.3,>=0.2.9, but you have langchain-core 0.1.52 which is incompatible.


In [2]:
!pip install langchain_google_genai

  Using cached langchain_core-0.2.9-py3-none-any.whl (321 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.52
    Uninstalling langchain-core-0.1.52:
      Successfully uninstalled langchain-core-0.1.52
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.20 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.2.9 which is incompatible.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.2.9 which is incompatible.


In [3]:
# !pip install langchain==0.1.0 langchain-google-genai==0.0.5 google-generativeai==0.3.2

In [4]:
import getpass
import os

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [6]:
google_api = os.environ["GOOGLE_API_KEY"] = getpass.getpass('Enter your google-api-key : ')

Enter your google-api-key : ··········


In [7]:
llm_gemini = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest",verbose = True,temperature = 0.1,google_api_key=google_api)

In [32]:
os.environ["SERPER_API_KEY"] =  getpass.getpass('Enter your Serper API key: ')

Enter your Serper API key: ··········


In [8]:
import os
from crewai import Agent, Task, Crew
from langchain.tools import DuckDuckGoSearchRun, Tool
from langchain.utilities import GoogleSerperAPIWrapper
# from llama_index import VectorStoreIndex, SimpleDirectoryReader
# from llama_index.tools import QueryEngineTool
# from langchain.tools import BrowserTool
from bs4 import BeautifulSoup
import requests


In [15]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool \

from crewai_tools import ScrapeElementFromWebsiteTool
search_tool = SerperDevTool()
scraper_tool = ScrapeWebsiteTool()
#data_extraction_tool = WebsiteSearchTool()
scrape_element_tool = ScrapeElementFromWebsiteTool()

In [25]:
researcher = Agent(
    llm=llm_gemini,
    role="Conduct thorough prior-art searches and ensure the invention is novel.",
    goal="Gather all relevant prior-art and document the novelty of the invention.",
    backstory="""With over 15 years of experience in intellectual property research, I specialize in conducting exhaustive prior-art searches. My expertise ensures comprehensive coverage of existing patents and technologies related to your invention. I am committed to uncovering innovative aspects that distinguish your invention in the competitive landscape, laying the groundwork for successful patenting.""",
    tools=[search_tool, scraper_tool, scrape_element_tool],
    verbose=True
)

patent_drafter = Agent(
    llm=llm_gemini,
    role="Draft the patent application including claims, abstract, and description.",
    goal="Create a well-structured and clear patent document.",
    backstory="""As a skilled patent drafter, I bring a decade of experience in crafting precise and comprehensive patent applications. My role is to transform technical details into a well-structured document that clearly defines the scope and uniqueness of your invention. I am dedicated to ensuring that every aspect of your invention is accurately represented, maximizing its patentability and protection.""",
    tools=[search_tool, scraper_tool, scrape_element_tool],
    verbose=True
)

patent_attorney = Agent(
    llm=llm_gemini,
    role="Provide legal advice and ensure the patent application complies with legal standards.",
    goal="Ensure the patent application is legally sound and enforceable.",
    backstory="""With a robust background in intellectual property law spanning two decades, I specialize in providing strategic legal counsel throughout the patenting process. My expertise lies in ensuring that patent applications meet rigorous legal standards and are strategically positioned for enforcement. I am dedicated to safeguarding your intellectual property rights and maximizing the commercial value of your inventions.""",
    tools=[search_tool, scraper_tool, scrape_element_tool],
    verbose=True
)



technical_specialist = Agent(
    llm=llm_gemini,
    role="Provide technical expertise and validate the technical accuracy of the patent application.",
    goal="Ensure all technical aspects of the patent are accurately represented.",
    backstory="""Drawing on over 12 years of experience in technology and engineering, I specialize in validating the technical content of patent applications. My role is crucial in ensuring that your invention's technical details are clearly and accurately articulated. I collaborate closely with patent attorneys and inventors to ensure that the application meets industry standards and regulatory requirements, enhancing its chances of successful approval.""",
    tools=[search_tool, scraper_tool, scrape_element_tool],
    verbose=True
)

patent_examiner_liaison = Agent(
    llm=llm_gemini,
    role="Communicate with the patent office and handle examination reports.",
    goal="Facilitate smooth communication with the patent office and address any issues raised.",
    backstory="""With a background in patent examination and administration, I excel in managing effective communication with patent offices worldwide. My expertise lies in handling examination reports and facilitating clear and prompt responses to patent office queries. I am committed to navigating the complexities of the patent examination process and advocating for the success of your patent application.""",
    tools=[search_tool, scraper_tool, scrape_element_tool],
    verbose=True
)

patent_filing_specialist = Agent(
    llm=llm_gemini,
    role="Handle the filing of the patent application and manage deadlines.",
    goal="Ensure timely filing and compliance with all procedural requirements.",
    backstory="""With extensive experience in patent filing across global jurisdictions, I specialize in managing efficient and compliant patent application submissions. My role involves meticulous attention to detail in preparing and submitting patent documents, ensuring adherence to regulatory requirements and deadlines. I am dedicated to providing a seamless filing experience, enabling inventors to protect their innovations effectively.""",
    tools=[search_tool, scraper_tool, scrape_element_tool],
    verbose=True
)


In [30]:
def create_tasks(product):

  researcher_task = Task(
      description=f"""Conduct a novelty search to find prior-art for the {product}.
      Conduct a comprehensive IP/Patent research for the product in the specified country.
        Use the following information: {product}
        1. Analyze existing patents and similar products in the market.
        2. Assess the novelty and non-obviousness of the invention.
        3. Identify potential legal challenges or conflicts.
        4. Evaluate market potential and competition.
        5. Research country-specific IP laws and requirements.

      """,
      expected_output="""A comprehensive report detailing all prior-art references and confirming the novelty of the invention.
        - Analysis of the product's novelty and patentability
        - Overview of relevant IP laws and filing requirements for the specified country
        - Market analysis and potential commercial value

      """,
      agent = researcher
  )

  patent_drafter_task = Task(
      description=f"""Write the patent application draft.
      Draft a comprehensive IP/Patent document for the product compliant with the specified country's laws.
        Use the following information: {product}
        1. Create a detailed technical description of the invention.
        2. Formulate clear and broad claims to protect the invention.
        3. Include diagrams or flowcharts if applicable.
        4. Ensure all aspects of the invention are covered.
        5. Adhere to the specific format required by the country's patent office.

      """,
      expected_output="""A complete patent application draft ready for review.,
      A complete IP/Patent draft including:
        - Title and abstract
        - Background of the invention
        - Detailed description of the product/process
        - Claims (independent and dependent)
        - Drawings or diagrams (if applicable)
        - Advantages and practical applications of the invention""",
      agent = patent_drafter,

      output_file = "draft_task.txt"
  )

  # patent_attorney_task = Task(
  #     description="Review the patent application for legal compliance.",
  #     expected_output="A legally compliant patent application."
  # )
  patent_attorney_task = Task(
    description="""Review the patent application for legal compliance.
    Ensure that the patent application adheres to all legal requirements and standards.
    Address potential legal challenges and maximize the protection of the invention.
    """,
    expected_output="""A legally compliant patent application that:
    - Meets all legal requirements and standards.
    - Addresses potential legal challenges or conflicts.
    - Maximizes the protection of the invention under applicable intellectual property laws.
    """,
    agent = patent_attorney
  )

  # technical_specialist_task = Task(
  #     description="Validate the technical content of the patent application.",
  #     expected_output="A technically accurate patent application."
  # )
  technical_specialist_task = Task(
    description="""Validate the technical content of the patent application.
    Verify that the technical description of the invention is accurate and detailed.
    Ensure that claims are supported by clear technical information.
    Collaborate to clarify technical aspects and improve the application.
    """,
    expected_output="""A technically accurate patent application that:
    - Provides a clear and detailed technical description of the invention.
    - Supports all claims with sufficient technical information.
    - Enhances the application’s chances of being granted based on technical merits.
    """,
    agent = technical_specialist
  )

  patent_examiner_liaison_task = Task(
      description=f"""Respond to the First Examination Report (FER).
      Thoroughly review and verify the IP/Patent document for the product.
        Use the following information: {input}
        1. Check for compliance with the specified country's patent laws.
        2. Ensure all claims are clear, concise, and fully supported.
        3. Verify that the document covers all aspects of the invention.
        4. Check for potential infringement issues.
        5. Suggest improvements or clarifications if necessary.""",
      expected_output=f"""A response to the FER addressing all raised objections.
      A verification report including:
        - Assessment of legal compliance
        - Analysis of claim strength and coverage
        - Identification of any potential weaknesses or areas for improvement
        - Suggestions for refining the patent document
        - Final recommendation (approve/revise/reject)""",
        output_file = "patent_examiner.txt",
      agent = patent_examiner_liaison
  )

  # patent_filing_specialist_task = Task(
  #     description="File the patent application with the appropriate patent office.",
  #     expected_output="A successfully filed patent application with confirmation receipt."
  # )
  patent_filing_specialist_task = Task(
    description="""File the patent application with the appropriate patent office.
    Manage the process from preparation to submission and tracking.
    Ensure compliance with filing requirements and monitor application progress.
    """,
    expected_output="""A successfully filed patent application with confirmation of receipt that:
    - Complies with all filing requirements and procedural guidelines.
    - Initiates the examination process by the patent office.
    - Provides a foundation for pursuing intellectual property rights based on the submitted application.
    """,
    agent = patent_filing_specialist
  )

  return [researcher_task, patent_drafter_task, patent_attorney_task, technical_specialist_task , patent_examiner_liaison_task , patent_filing_specialist_task ]

In [33]:

def Create_IP(product):
    tasks = create_tasks(product)
    crew = Crew(
        agents=[patent_drafter,patent_drafter,  patent_attorney, technical_specialist, patent_examiner_liaison, patent_filing_specialist],
        tasks=tasks,
        verbose=2,
        # max_rpm = 29,
        process_timeout=600
    )
    result = crew.kickoff()
    return result

def main():
    product = input("ENTER WHAT IP YOU WANT : ")
    result = Create_IP(product)

    print("IP RESULT :")
    print(result)


if __name__ == "__main__":
    main()

 [2024-06-25 16:40:13][DEBUG]: == Working Agent: Conduct thorough prior-art searches and ensure the invention is novel.
 [2024-06-25 16:40:13][INFO]: == Starting Task: Conduct a novelty search to find prior-art for the Umbrella which can shift shape based on the size of user if user is thin then umbrella takes up less space and if user is heavy then it takes more space.
      Conduct a comprehensive IP/Patent research for the product in the specified country.
        Use the following information: Umbrella which can shift shape based on the size of user if user is thin then umbrella takes up less space and if user is heavy then it takes more space
        1. Analyze existing patents and similar products in the market.
        2. Assess the novelty and non-obviousness of the invention.
        3. Identify potential legal challenges or conflicts.
        4. Evaluate market potential and competition.
        5. Research country-specific IP laws and requirements.
      
      


> Entering

Thought: While I'm finding some information on sensors, the search isn't giving me specific prior art related to umbrellas. I should ask a coworker to analyze these search results for relevant patents and assess the novelty of the umbrella concept. 
Action: Delegate work to coworker
Action Input: {"coworker": "Provide technical expertise and validate the technical accuracy of the patent application.", "task": "Analyze the following search results to identify any relevant prior art related to a size-adjusting umbrella that expands based on the user's size. The umbrella would theoretically use sensors to detect the user's size and adjust the canopy accordingly.  Assess the novelty of this concept based on your findings and identify any potential legal challenges. The search results are based on queries for patents and products related to size-adjusting umbrellas, weight and pressure sensors in fabric, and sensors that can detect body size. Here are the search results: {Observation}", "co

 


Search results: Title: US9538819B1 - Umbrella having an expandable canopy
Link: https://patents.google.com/patent/US9538819B1/en
Snippet: Search within the title, abstract, claims, or full patent document: You can restrict your search to a specific field using field names. Use TI= to search in the ...
---
Title: Umbrella-type Patents (Class 135/98)
Link: https://patents.justia.com/patents-by-us-classification/135/98?page=6
Snippet: Search for Umbrella-type Patents and Patent Applications (Class 135/98) Filed with the USPTO.
---
Title: Class 135 TENT, CANOPY, UMBRELLA, OR CANE - USPTO
Link: https://www.uspto.gov/web/offices/ac/ido/oeip/taf/def/135.htm
Snippet: SEE OR SEARCH THIS CLASS, SUBCLASS: 15.1 for umbrellas of paper or similar fragile material, but completely assembled in folded-about-the-stick- ...
---
Title: Cantilever umbrella - Patent 2845510
Link: https://data.epo.org/publication-server/rest/v1.2/patents/EP2845510NWA1/document.html
Snippet: [0001] The application relates

Thought: I need to analyze the provided search results for relevance to size-adjusting umbrellas and the use of sensors.
Action: Read website content
Action Input: {"website_url": "https://patents.google.com/patent/US9538819B1/en"} 

US9538819B1 - Umbrella having an expandable canopy - Google Patents
US9538819B1 - Umbrella having an expandable canopy - Google Patents
Umbrella having an expandable canopy 
Download PDF
Info
Publication number
US9538819B1
US9538819B1
US15/132,407
US201615132407A
US9538819B1
US 9538819 B1
US9538819 B1
US 9538819B1
US 201615132407 A
US201615132407 A
US 201615132407A
US 9538819 B1
US9538819 B1
US 9538819B1
Authority
US
United States
Prior art keywords
canopy
umbrella
rib
runner
post
Prior art date
2016-04-19
Legal status (The legal status is an assumption and is not a legal conclusion. Google has not performed a legal analysis and makes no representation as to the accuracy of the status listed.)
Expired - Fee Related
Application number
US15/132,407
Inventor


Thought: The previous attempt failed because I reused the same URL. I need to analyze the content of the patent to see if it's relevant to a size-adjusting umbrella.
Action: Read website content
Action Input: {"website_url": "https://patents.google.com/patent/US9538819B1/en", "css_element": "div.description.content"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.





Thought: The previous attempt failed because I reused the same URL. I need to analyze the content of the patent to see if it's relevant to a size-adjusting umbrella.
Action: Read website content
Action Input: {"website_url": "https://patents.google.com/patent/US9538819B1/en", "css_element": "section.abstract"} 

US9538819B1 - Umbrella having an expandable canopy - Google Patents
US9538819B1 - Umbrella having an expandable canopy - Google Patents
Umbrella having an expandable canopy 
Download PDF
Info
Publication number
US9538819B1
US9538819B1
US15/132,407
US201615132407A
US9538819B1
US 9538819 B1
US9538819 B1
US 9538819B1
US 201615132407 A
US201615132407 A
US 201615132407A
US 9538819 B1
US9538819 B1
US 9538819B1
Authority
US
United States
Prior art keywords
canopy
umbrella
rib
runner
post
Prior art date
2016-04-19
Legal status (The legal status is an assumption and is not a legal conclusion. Google has not performed a legal analysis and makes no representation as to the accuracy of the



> Entering new CrewAgentExecutor chain...
Thought: I need to understand the existing technology landscape related to the umbrella's proposed features. This includes searching for size-adjusting mechanisms, pressure sensors in fabric, and body size detection technologies. 
Action: Search the internet
Action Input: {"search_query": "Patents for umbrellas with adjustable canopy size"}  


Search results: Title: US9538819B1 - Umbrella having an expandable canopy
Link: https://patents.google.com/patent/US9538819B1/en
Snippet: Search specific patents by importing a CSV or list of patent ... The umbrella having an expandable canopy is an umbrella with a canopy of adjustable size.
---
Title: US8104492B2 - Adjustable offset umbrella - Google Patents
Link: https://patents.google.com/patent/US8104492B2/en
Snippet: An adjustable offset umbrella has a generally vertical main pole and a sliding member slidably mounted to the main pole.
---
Title: Umbrella Patents and Patent Applications (Class 135

Thought: This patent focuses on an adjustable offset mechanism for tilting the canopy, not on adjusting the canopy size based on user size. I should continue analyzing the remaining search results.
Action: Read website content
Action Input: {"website_url": "https://patents.justia.com/patents-by-us-classification/135/15.1"}  

Umbrella Patents and Patent Applications (Class 135/15.1) - Justia Patents Search
Log In
Sign Up
Find a Lawyer
Ask a Lawyer
Research the Law
Law Schools
Laws & Regs
Newsletters
Marketing Solutions
Justia Connect
Pro Membership
Basic Membership
Justia Lawyer Directory
Platinum Placements
Gold Placements
Justia Elevate
SEO
Websites
Blogs
Justia Amplify
PPC Management
Google Business Profile
Social Media
Justia Onward Blog
Justia
Patents
Tent, Canopy, Umbrella, Or Cane Patents Umbrella Patents (Class 135/15.1)
 Umbrella Patents (Class 135/15.1)
Combined (Class 135/16)
 With cane (Class 135/17)
 With imitation cane case (Class 135/18)
 With independent stick (Class 13

KeyboardInterrupt: 

In [34]:
info_gathering_agent = Agent(
    llm=llm_gemini,
    role="Product Information Specialist",
    goal="Gather comprehensive and relevant information about the product for IP/Patent filing",
    backstory="""You are an expert in product analysis and IP preparation. Your role is to ask
    insightful questions to gather all necessary information about a product for patent filing.""",
    allow_delegation=False,
    verbose=True,
)




In [37]:
def dynamic_info_gathering():
    print("\nOur AI will now ask you some questions about your product/invention.")
    product_info = {}

    initial_task = Task(
        description="Ask the user for the name of their product and the country where they want to file the patent.",
        expected_output="A question asking for the product name and filing country.",
        agent=info_gathering_agent
    )

    crew = Crew(
        agents=[info_gathering_agent],
        tasks=[initial_task],
        verbose=2
    )

    initial_response = crew.kickoff()
    print("\nAI: " + initial_response)
    user_input = input("Your response: ")
    product_info['initial_info'] = user_input

    while True:
        follow_up_task = Task(
            description=f"""Based on the user's response: '{user_input}', ask another relevant
            question to gather more details about the product. If you have sufficient information,
            say 'Thank you, that's all the information I need.' and end the questioning.
            if {user_input == '/thatsall'} quit your task and give this  result further
            """,
            expected_output="Either a follow-up question or a statement indicating the end of questioning.",
            agent=info_gathering_agent
        )

        crew = Crew(
            agents=[info_gathering_agent],
            tasks=[follow_up_task],
            verbose=2
        )

        response = crew.kickoff()
        print("\nAI: " + response)

        if "thank you" in response.lower() and "that's all" in response.lower():
            break

        user_input = input("Your response: ")
        product_info[f'info_{len(product_info)}'] = user_input

    return product_info

In [38]:

def Create_IP(product):
    product_info = dynamic_info_gathering()
    tasks = create_tasks(product_info)
    crew = Crew(
        agents=[patent_drafter,patent_drafter,  patent_attorney, technical_specialist, patent_examiner_liaison, patent_filing_specialist],
        tasks=tasks,
        verbose=2,
        # max_rpm = 29,
        process_timeout=600
    )
    result = crew.kickoff()
    return result

def main():
    product = input("ENTER WHAT IP YOU WANT : ")
    result = Create_IP(product)

    print("IP RESULT :")
    print(result)


if __name__ == "__main__":
    main()

ENTER WHAT IP YOU WANT : Smart Recycling Bin with AI Sorting



Our AI will now ask you some questions about your product/invention.
 [2024-06-25 16:56:40][DEBUG]: == Working Agent: Product Information Specialist
 [2024-06-25 16:56:40][INFO]: == Starting Task: Ask the user for the name of their product and the country where they want to file the patent.


> Entering new CrewAgentExecutor chain...
Thought: I can ask for the product name and filing country in a clear and concise way to ensure I get the information needed. 

Final Answer:  What is the name of the product you would like to patent, and in which country do you intend to file the patent? 


> Finished chain.
 [2024-06-25 16:56:42][DEBUG]: == [Product Information Specialist] Task output: What is the name of the product you would like to patent, and in which country do you intend to file the patent?



AI: What is the name of the product you would like to patent, and in which country do you intend to file the patent?
Your response: SmartBinAI and in India


 [2024-06-25 16:57:06][DEBUG]: == Working Agent: Product Information Specialist
 [2024-06-25 16:57:06][INFO]: == Starting Task: Based on the user's response: 'SmartBinAI and in India', ask another relevant
            question to gather more details about the product. If you have sufficient information,
            say 'Thank you, that's all the information I need.' and end the questioning.
            if False quit your task and give this  result further 
            


> Entering new CrewAgentExecutor chain...
Thought: I need to understand what SmartBinAI is and how it functions in the Indian context before I can determine if I have enough information for an IP filing. 
Final Answer: "Can you please describe the problem that SmartBinAI solves in the Indian market and its intended application?" 


> Finished chain.
 [2024-06-25 16:57:08][DEBUG]: == [Product Information Specialist] Task output: "Can you please describe the problem that SmartBinAI solves in the Indian market and its int

 [2024-06-25 16:57:34][DEBUG]: == Working Agent: Product Information Specialist
 [2024-06-25 16:57:34][INFO]: == Starting Task: Based on the user's response: 'In India, waste management is a critical issue that impacts public health, the environment, and urban infrastructure. Despite various initiatives to promote recycling, several challenges persist', ask another relevant
            question to gather more details about the product. If you have sufficient information,
            say 'Thank you, that's all the information I need.' and end the questioning.
            if False quit your task and give this  result further 
            


> Entering new CrewAgentExecutor chain...
Thought: I need to understand what kind of product or solution the user is envisioning to address the challenges in waste management in India. Asking about the specific approach will help me determine if there's a potentially patentable invention. 

Final Answer: "What specific approach or solution does your p

 [2024-06-25 16:58:21][DEBUG]: == Working Agent: Product Information Specialist
 [2024-06-25 16:58:21][INFO]: == Starting Task: Based on the user's response: 'SmartBinAI offers an AI-powered sorting mechanism that automatically identifies and segregates recyclables, reducing contamination. It provides real-time user feedback for correct disposal practices, collects data for optimized waste management, and integrates with smart city infrastructure for efficient waste collection and timely maintenance alerts.', ask another relevant
            question to gather more details about the product. If you have sufficient information,
            say 'Thank you, that's all the information I need.' and end the questioning.
            if False quit your task and give this  result further 
            


> Entering new CrewAgentExecutor chain...
Thought: I need to understand more about the AI-powered sorting mechanism. What kind of data does it use to make decisions? 

Final Answer: "Can you ela

...

 [2024-06-25 17:03:24][DEBUG]: == Working Agent: Product Information Specialist
 [2024-06-25 17:03:24][INFO]: == Starting Task: Based on the user's response: '/thatsall please stop it ', ask another relevant
            question to gather more details about the product. If you have sufficient information,
            say 'Thank you, that's all the information I need.' and end the questioning.
            if False quit your task and give this  result further 
            


> Entering new CrewAgentExecutor chain...


Thought: The user seems to be indicating they don't want to provide more information. It's important to respect that and not pry further. I should end the questioning gracefully. 
Final Answer: Thank you, that's all the information I need. 


> Finished chain.
 [2024-06-25 17:03:26][DEBUG]: == [Product Information Specialist] Task output: Thank you, that's all the information I need.



AI: Thank you, that's all the information I need.
 [2024-06-25 17:03:26][DEBUG]: == Working Agent: Conduct thorough prior-art searches and ensure the invention is novel.
 [2024-06-25 17:03:26][INFO]: == Starting Task: Conduct a novelty search to find prior-art for the {'initial_info': 'SmartBinAI and in India', 'info_1': 'In India, waste management is a critical issue that impacts public health, the environment, and urban infrastructure. Despite various initiatives to promote recycling, several challenges persist', 'info_2': 'SmartBinAI offers an AI-powered sorting mechanism that automatically identifie

 


Search results: Title: The waste segregation method using machine learning technique
Link: https://patents.google.com/patent/AU2021101744A4/en
Snippet: The proposed trash bin is equipped with sensor system which is able to segregate the waste in an intelligent way along with report generation of waste collected ...
---
Title: US20070158253A1 - Waste collection system - Google Patents
Link: https://patents.google.com/patent/US20070158253A1/en
Snippet: A transportable waste collection system for collecting waste found in a body of flowing water, including non-navigable waters, the system having a flotation ...
---
Title: Artificial intelligence for waste management in smart cities: a review
Link: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10169138/
Snippet: Artificial intelligence allows for identifying and sorting waste with an accuracy ranging from 72.8 to 99.95%. Artificial intelligence combined ...
---
Title: Victor Dewulf and Peter Hedley | Epo.org
Link: https://www.epo.org/en

Thought: The Australian patent seems relevant. I should read through it to understand its claims and how similar it is to SmartBinAI.
Action: Read website content
Action Input: {"website_url": "https://patents.google.com/patent/AU2021101744A4/en"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.





KeyboardInterrupt: 